<a href="https://colab.research.google.com/github/RitwickJoshi/Colaboratory/blob/master/Cat_Dog_Detector_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
Date: 25-12-2020
filename: Cats_Dogs_Detector_v2.ipynb
@author: Ritwick Joshi
program name: Cat and Dog Classification
program description: objective of the program is to come up with accuracy of cat-dog-detection-model based on CNN classification
"""

'\nDate: 25-12-2020\nfilename: Cats_Dogs_Detector_v1.ipynb\n@author: Ritwick Joshi\nprogram name: Cat and Dog Classification\nprogram description: objective of the program is to come up with accuracy of cat-dog-detection-model based on CNN classification\n'

In [6]:
#importing needed modules (if not fulfilled more are imported later)
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

#directory where the dataset is located
main_directory = r'/content/drive/MyDrive/Colab_Notebooks/Dataset'
train_directory = os.path.join(main_directory, 'training')
test_directory = os.path.join(main_directory, 'test')

#creating a iterator with specified set of rules that is to be performed on the image
#this is done so as to augment the data
training_generator = ImageDataGenerator(
    width_shift_range=0.15, 
    height_shift_range=0.15,
    rescale = 1./255,
    horizontal_flip = True,
    shear_range = 0.10,
    rotation_range=20,
)

testing_generator = ImageDataGenerator(
    rescale = 1./255
)

#created a variable with 20 batches of data for a little ease of computation
trainer = training_generator.flow_from_directory(
    train_directory, 
    batch_size = 20,
    target_size = (200, 200), 
    class_mode = 'binary',
)
tester = testing_generator.flow_from_directory(
    test_directory,
    batch_size = 20,
    target_size = (200, 200),
    class_mode = 'binary',
)



Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [12]:
#model selection and addition of convolutions
model = Sequential()

model.add(Conv2D(32, (3,3), kernel_initializer='he_normal', activation='relu', input_shape=(200,200,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))


model.add(Flatten())

model.add(Dense( 
    128, 
    activation='relu', 
    input_shape = (200,200,3),
    kernel_initializer='he_normal'
))

model.add(Dropout(.5))

model.add(Dense(
    1,
    activation='sigmoid'
))

In [13]:
#compilation and learning rate setting of the model
from keras.optimizers import RMSprop
opti = RMSprop(lr = 1e-3)
model.compile(
    optimizer = opti,
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
#training the model based on the given neurons and epochs
result = model.fit_generator(
    trainer,
    epochs = 400,
    validation_data = tester,
    steps_per_epoch = 100,
    validation_steps = 50
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/400
100/100 [==============================] - 634s 6s/step - loss: 0.7989 - accuracy: 0.5251 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 2/400
100/100 [==============================] - 166s 2s/step - loss: 0.6962 - accuracy: 0.4937 - val_loss: 0.6781 - val_accuracy: 0.6490
Epoch 3/400
100/100 [==============================] - 166s 2s/step - loss: 0.6807 - accuracy: 0.5889 - val_loss: 0.7502 - val_accuracy: 0.6040
Epoch 4/400
 82/100 [=======================>......] - ETA: 26s - loss: 0.6492 - accuracy: 0.6257

In [ ]:
#accuracy and loss variables
acc = result.history['accuracy']
vacc = result.history['val_accuracy']
repeated_for = range(1,len(acc)+1)

loss = result.history['loss']
vloss = result.history['val_loss']
model.summary()

In [ ]:
#plotting the graph of training and validation accuracy

plt.plot(repeated_for, acc, 'go', label = 'Training ACC')
plt.plot(repeated_for, vacc, 'b', label = 'Validation ACC')
plt.title('Training and Validation Accuracy')
plt.legend()


In [ ]:
#plotting the graph of training and validation loss

plt.plot(repeated_for, loss, 'go', label = 'Training LOSS')
plt.plot(repeated_for, vloss, 'b', label = 'Validation LOSS')
plt.title('Training and Validation Loss')
plt.legend()



In [ ]:
#saving the model
model.save(r'/content/drive/MyDrive/Colab_Notebooks/Dataset/test_model_400_epochs_4.h5')


In [ ]:
#printing the accuracy of the model
#load the model in the google drive or change the path

from keras.models import load_model

md = load_model(r'/content/drive/MyDrive/Colab_Notebooks/Dataset/test_model_400_epochs_4.h5')

_,accuracy_model = md.evaluate_generator(tester, steps=len(tester), verbose=0)
print(f'{accuracy_model}')

In [ ]:
from keras.preprocessing import image
import numpy as np

img_path = (r'/content/drive/MyDrive/Colab_Notebooks/Dataset/training/cats/cat.999.jpg')
img = image.load_img(img_path, target_size=(200,200))
plt.imshow(img)
img = np.expand_dims(img, axis=0)
result=md.predict_classes(img)
plt.title((result[0][0]))
plt.show()